In [1]:
from cmath import nan
from PIL import Image
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
import os, glob
import cv2
import pandas as pd
from typing import Tuple
from PIL import Image

In [2]:
'''crop and rotate operation for a single image'''
def crop_rotate(filename, angle = -1.5, left = 135, upper =80, right = 600, lower = 390):
    with Image.open(filename) as img:
         # (left, upper, right, lower) = (100, 60, 630, 400)
        rotated = img.rotate(angle, expand = 1)
    return rotated.crop((left, upper, right, lower))

"""
exclude wells with black background (idx start from 0)
"""
def exclude_wells(col_idx, row_idx):
    if ((col_idx == 2 or col_idx == 3) and row_idx == 12) or ((row_idx ==0 or row_idx == 15) and (col_idx==11 or col_idx ==12)) or (row_idx == 2 and col_idx==20):
        return True
    return False
"""
input: img or preprocessed img
output: conceptually a matrix of 16 x 24, each entry is the isolated well image
"""
def isolate_each_well(img_th):
    to_return = []
    y = img_th.shape[0] / 16
    x = img_th.shape[1] / 24
    for row in np.arange(0, img_th.shape[0], y):
        col_out = []
        for col in np.arange(0, img_th.shape[1], x):
            col_out.append(img_th[round(row):round(row+y),round(col):round(col+x)])
        to_return.append(col_out)
    return np.array(to_return, dtype=object)

letters_to_index = {
        "A": 0,
        "B": 1,
        "C": 2,
        "D": 3,
        "E": 4,
        "F": 5,
        "G": 6,
        "H": 7,
        "I": 8,
        "J": 9,
        "K": 10,
        "L": 11,
        "M": 12,
        "N": 13,
        "O": 14,
        "P": 15
    }

"""
modify the dataframe loaded from 'xxx_labels.csv"
split 'destination_well' and add column 'row_id' and 'col_id'
"""
def letter2index_df(df):
    rows = []
    cols = []
    destination_wells = df['destination_well'].values
    for destination in destination_wells:
        row = letters_to_index[destination[0]]
        col = int(destination[1:]) - 1
        rows.append(row)
        cols.append(col)

    df['row_id'] = rows
    df['col_id'] = cols


In [4]:
'''
make new csv 'xxx_labels_new.csv'
for folder in glob.glob(path+'/*/'):
    print(folder)
    for csv in glob.glob(folder +'/*.csv'):
        print(csv)
        labels = pd.read_csv(csv, index_col=0)
        letter2index_df(labels)
        csv_name = csv.split('.')[0]
        csv_path = os.path.join(csv_name+'_new.csv')
        print(csv_path)
        labels.to_csv(csv_path)
'''

"\nmake new csv 'xxx_labels_new.csv'\nfor folder in glob.glob(path+'/*/'):\n    print(folder)\n    for csv in glob.glob(folder +'/*.csv'):\n        print(csv)\n        labels = pd.read_csv(csv, index_col=0)\n        letter2index_df(labels)\n        csv_name = csv.split('.')[0]\n        csv_path = os.path.join(csv_name+'_new.csv')\n        print(csv_path)\n        labels.to_csv(csv_path)\n"

In [12]:
# path = r'revised_train_221027'
# wells_dir = r'train_221027'
path = r'revised_val_small3_221027'
wells_dir = r'val3_221027'
well_img_id = 0
out_labels = []

In [15]:
for folder in glob.glob(path+'/*/'):
    # print(folder)
    barcode = folder.split('/')[1]
    print(barcode)
    for csv in glob.glob(folder +'/*_new.csv'):
        # print(csv)
        labels = pd.read_csv(csv, index_col=0)
    for img_name in glob.glob(folder +'/*.jpg'):
        # print(img_name)
        img = crop_rotate(img_name)
        # img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
        # plt.imshow(img, cmap = 'gray')
        wells = isolate_each_well(img)
        # plt.imshow(wells[0][0], cmap = 'gray')
        for row in range(wells.shape[0]):
            for col in range(wells.shape[1]):
                if not exclude_wells(col, row):  # exclude wells with black background
                    target = labels.loc[(labels['col_id']==col) & (labels['row_id']==row),'label'].tolist()[0]
                    if not np.isnan(target):
                        '''save each well_img'''
                        path_save_well_img = os.path.join(wells_dir, f'{well_img_id}.jpg')
                        well = np.array(wells[row,col])
                        img = Image.fromarray(well)
                        img.save(path_save_well_img)
                        '''save each label in csv'''
                        info = [well_img_id, barcode, row, col, target]
                        info = np.asarray(info)
                        out_labels.append(info)
                        well_img_id += 1


1044837
1036688
1035231
1044836
1036687
1035230
1040848
1048040
1042117
1042116
1042118
1043496
1048041
1044835
1046898
1046899
1046897
1048044
1043494
1048043
1035229
1042115
1043495
1048042
1035228


In [17]:
csv = np.vstack(out_labels)
df = pd.DataFrame(csv)
df.columns = ["data_id", "plate_path", "row_id", "col_id", "label"]
# labels_file_name = 'labels_train_221027.csv'
labels_file_name = 'labels_val_221027.csv'
df.to_csv(labels_file_name, index=False)
df = pd.read_csv(labels_file_name)